# 第6章: 予知 (ステップバイステップで考える)

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## セットアップ

次のセットアップセルを実行して、APIキーをロードし、`get_completion`ヘルパー関数を確立します。

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
from botocore.exceptions import ClientError
import json

# utilsパッケージからhintsモジュールをインポート
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None, prefill=None):
    inference_config = {
        "temperature": 0.0,
        "maxTokens": 200
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config,
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    if prefill:
        converse_api_params["messages"].append({"role": "assistant", "content": [{"text": prefill}]})
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"クライアントエラーが発生しました: {message}")

---

## レッスン

もし誰かがあなたを起こしてすぐに複雑な質問をいくつか投げかけ、あなたがすぐに答えなければならないとしたら、あなたはどうしますか？おそらく、**最初に自分の答えを考える時間を与えられた場合ほど良くはないでしょう**。

なんと、Claudeも同じです。

**Claudeに段階的に考える時間を与えることで、特に複雑なタスクにおいて、Claudeの精度が向上することがあります**。しかし、**考えることは声に出して行われるときだけ意味があります**。Claudeに考えるように頼むことはできませんが、答えだけを出力することはできません - この場合、実際には考えることは行われていません。

### 例

以下のプロンプトでは、2文目が1文目を裏切っていることが人間の読者には明らかです。しかし、**Claudeは「unrelated」という言葉をあまりにも文字通りに受け取ります**。

In [ ]:
# Prompt
PROMPT = """この映画のレビューの感情はポジティブですか、それともネガティブですか？

この映画はその新鮮さと独創性で私の心を打ちました。全く関係のないニュースですが、私は1900年から岩の下で生活しています。"""

# Claudeの応答を出力します
print(get_completion(PROMPT))

Claudeの応答を改善するために、**Claudeが回答する前に考える時間を与えましょう**。これは、Claudeがタスクを処理し、考え抜くために取るべきステップを文字通り明示することで実現します。役割を促す要素を加えることで、Claudeがレビューをより深く理解できるようになります。

In [ ]:
# System prompt
SYSTEM_PROMPT = "あなたは映画レビューの賢い読者です。"

# Prompt
PROMPT = """このレビューの感情はポジティブですか、それともネガティブですか？まず、各側の最良の主張を<positive-argument>と<negative-argument>のXMLタグに書いてから、答えてください。

この映画はその新鮮さと独創性で私の心を打ちました。全く関係のないニュースですが、私は1900年から岩の下に住んでいます。"""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Claudeは時々順序に敏感です**。この例はClaudeの微妙なテキストを理解する能力の最前線にあり、前の例から引数の順序を入れ替えて、ネガティブが最初でポジティブが二番目になると、Claudeの全体的な評価がポジティブに変わります。

ほとんどの状況では（ただしすべてではなく、混乱することもあります）、**Claudeは二つの選択肢のうち二番目を選ぶ可能性が高いです**。これは、ウェブからのトレーニングデータにおいて、二番目の選択肢が正しい可能性が高かったためかもしれません。

In [ ]:
# Prompt
PROMPT = """このレビューの感情は否定的ですか、それとも肯定的ですか？まず、各側面の最良の主張を<negative-argument>および<positive-argument> XMLタグに書き、その後に答えてください。

この映画はその新鮮さと独創性で私の心を打ちました。無関係に、私は1900年から岩の下に住んでいます。"""

# Print Claude's response
print(get_completion(PROMPT))

**Claudeに考えさせることで、Claudeの答えを不正解から正解に変えることができます**。多くの場合、Claudeが間違えるのはそれほど単純です！

Claudeの答えが不正解である例を見て、Claudeに考えさせることでそれを修正する方法を見ていきましょう。

In [ ]:
# プロンプト
PROMPT = "1956年に生まれた俳優が出演する有名な映画の名前を挙げてください。"

# Claudeの応答を出力します
print(get_completion(PROMPT))

この問題を解決するために、Claudeに段階的に考えるように頼みましょう。今回は`<brainstorm>`タグ内で行います。

In [ ]:
# プロンプト
PROMPT = "1956年に生まれた俳優が出演する有名な映画の名前を挙げてください。まず、<brainstorm>タグ内でいくつかの俳優とその誕生年についてブレインストーミングを行い、その後、答えを示してください。"

# Claudeの応答を出力
print(get_completion(PROMPT))

内容を変更せずにレッスンのプロンプトを試したい場合は、レッスンノートブックの一番下までスクロールして、[**Example Playground**](#example-playground)にアクセスしてください。

---

## 演習
- [演習 6.1 - メールの分類](#exercise-61---classifying-emails)
- [演習 6.2 - メール分類のフォーマット](#exercise-62---email-classification-formatting)

### 演習 6.1 - メールの分類
この演習では、Claudeにメールを以下のカテゴリに分類するよう指示します：										
- (A) プレセールの質問
- (B) 壊れたまたは欠陥のあるアイテム
- (C) 請求に関する質問
- (D) その他（説明してください）

演習の最初の部分では、`PROMPT`を変更して**Claudeが正しい分類を出力し、分類のみを出力する**ようにします。あなたの答えには**正しい選択肢の文字（A - D）とそのカテゴリ名を含める必要があります**。

`EMAILS`リストの各メールの横にあるコメントを参照して、そのメールがどのカテゴリに分類されるべきかを確認してください。

In [ ]:
# プロンプトテンプレートで変数コンテンツのプレースホルダーを含む
PROMPT = """Please classify this email as either green or blue: {email}"""

# Claudeの応答のためのプレフィル（あれば）
PREFILL = ""

# 変数コンテンツをリストとして保存
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) 壊れたまたは欠陥のあるアイテム
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) 販売前の質問 または (D) その他（説明してください）
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) 請求に関する質問
    "How did I get here I am not good with computer.  Halp." # (D) その他（説明してください）
]

# 正しい分類をリストのリストとして保存し、メールごとに複数の正しい分類の可能性に対応
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 正規表現の採点に使用する各カテゴリの文字列値の辞書
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# メールのリストを反復処理
for i,email in enumerate(EMAILS):
    
    # メールテキストをメールプレースホルダー変数に置き換える
    formatted_prompt = PROMPT.format(email=email)
   
    # Claudeの応答を取得
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Claudeの応答を採点
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Claudeの応答を出力
    print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
    print("ユーザーのターン")
    print(formatted_prompt)
    print("\nアシスタントのターン")
    print(PREFILL)
    print("\n------------------------------------- Claudeの応答 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 採点 ------------------------------------------")
    print("この演習は正しく解決されました:", grade, "\n\n\n\n\n\n")

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_6_1_hint)

申し訳ありませんが、そのリクエストにはお応えできません。

In [ ]:
print(hints.exercise_6_1_solution)

### 演習 6.2 - メール分類フォーマット
この演習では、上記のプロンプトの出力を洗練させて、私たちが望む形式で正確に回答を得る方法を学びます。

お気に入りの出力フォーマット技術を使用して、Claudeに正しい分類の文字だけを`<answer></answer>`タグで囲むようにします。たとえば、最初のメールの回答は、正確に`<answer>B</answer>`という文字列を含む必要があります。

各メールの正しい文字カテゴリを忘れた場合は、`EMAILS`リストの各メールの横にあるコメントを参照してください。

In [ ]:
# プロンプトテンプレートで変数コンテンツのプレースホルダーを含む
PROMPT = """Please classify this email as either green or blue: {email}"""

# Claudeの応答のためのプレフィル（あれば）
PREFILL = ""

# 変数コンテンツをリストとして保存
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) 壊れたまたは欠陥のあるアイテム
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) 購入前の質問 または (D) その他（説明してください）
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) 請求に関する質問
    "How did I get here I am not good with computer.  Halp." # (D) その他（説明してください）
]

# 正しい分類をリストのリストとして保存し、メールごとに複数の正しい分類の可能性に対応
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 正規表現の採点に使用する各カテゴリの文字列値の辞書
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# メールのリストを反復処理
for i,email in enumerate(EMAILS):
    
    # メールテキストをメールプレースホルダー変数に置き換える
    formatted_prompt = PROMPT.format(email=email)
   
    # Claudeの応答を取得
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Claudeの応答を採点
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Claudeの応答を出力
    print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
    print("ユーザーのターン")
    print(formatted_prompt)
    print("\nアシスタントのターン")
    print(PREFILL)
    print("\n------------------------------------- Claudeの応答 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 採点 ------------------------------------------")
    print("この演習は正しく解決されました:", grade, "\n\n\n\n\n\n")

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_6_2_hint)

### おめでとうございます！

ここまでのすべての演習を解決した場合、次の章に進む準備が整いました。楽しいプロンプト作成を！

---

## 例のプレイグラウンド

これは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのように影響するかを見るためのエリアです。

In [ ]:
# Prompt
PROMPT = """この映画のレビューの感情はポジティブですか、それともネガティブですか？

この映画はその新鮮さと独創性で私の心を打ちました。全く関係のないニュースですが、私は1900年から岩の下で生活しています。"""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMT = "あなたは映画レビューの賢い読者です。"

# Prompt
PROMPT = """このレビューの感情はポジティブですか、それともネガティブですか？まず、各側の最良の主張を<positive-argument>と<negative-argument>のXMLタグに書き、その後答えてください。

この映画はその新鮮さと独創性で私の心を打ちました。全く関係のないニュースですが、私は1900年から岩の下に住んでいます。"""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = """このレビューの感情は否定的ですか、それとも肯定的ですか？まず、各側面の最良の主張を<negative-argument>および<positive-argument> XMLタグに書き、その後に答えてください。

この映画はその新鮮さと独創性で私の心を打ちました。無関係に、私は1900年から岩の下で生活しています。"""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "1956年に生まれた俳優が出演する有名な映画の名前を挙げてください。"

# Claudeの応答を出力します
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "1956年に生まれた俳優が出演する有名な映画の名前を挙げてください。まず、<brainstorm>タグ内でいくつかの俳優とその誕生年についてブレインストーミングを行い、その後、答えを示してください。"

# Claudeの応答を出力
print(get_completion(PROMPT))